## Library imports

In [3]:
import cv2
import pandas as pd
from sort.sort import *
from classify_by_movement import *
from calculate_features import *

In [4]:
# Show graph inline
%matplotlib inline

## Show results of video

In [11]:
# Open the video file
video_number = str(11)
fps = 50  # Replace with your video's frame rate

#video_path = 'data/data_orig/val/'+video_number+'/'+video_number+'.mp4'
video_path = '../data/ViSEM_Tracking_extended/train/'+ video_number + '/' + video_number + '.mp4'

In [ ]:
cap = cv2.VideoCapture(video_path)
print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
# 

tracking = pd.read_csv('../results/data_sperm_tracking/sperm_tracking_data_' + video_number + '_30s_15_20_01.csv')

frame_id = 0
trajectories = {}


while cap.isOpened():
    ret, frame = cap.read()
    
    tracks = tracking[tracking['frame_id'] == frame_id]
    print(frame_id)
    print(len(tracks))
    for _, row in tracks.iterrows():
        cx, cy = int(row['cx']), int(row['cy'])
        
        # Set sperm trajectory
        track_id = int(row['track_id'])
        
        if track_id not in trajectories:
            trajectories[track_id] = []
        trajectories[track_id].append((cx, cy))
        
        # Draw path
        for i in range(1, len(trajectories[track_id])):
            cv2.line(frame, (int(trajectories[track_id][i - 1][0]),int(trajectories[track_id][i - 1][1])), (int(trajectories[track_id][i][0]),int(trajectories[track_id][i][1])), (0,0,0), 1)

        
    # Wait for a key press (25ms delay between frames)
    # Press 'q' to exit the sequence
    cv2.imshow('YOLOv5 Inference', frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
    
    frame_id += 1
# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()

1470
0
44
1
44
2
44
3
44
4
44
5
44
6
44
7
44
8
44
9
44
10
44
11
44
12
44
13
44
14
44
15
44
16
44
17
44
18
44
19
44
20
44
21
44
22
44
23
44
24
44
25
44
26
43
27
43
28
43
29
43
30
43
31
43
32
43
33
42
34
42
35
42
36
42
37
42
38
42
39
42
40
42
41
42
42
42
43
42
44
42
45
42
46
42
47
42
48
41
49
41
50
41
51
40
52
40
53
40
54
40
55
40
56
40
57
40
58
40
59
40
60
40
61
40
62
40
63
40
64
40
65
40
66
39
67
39
68
39
69
39
70
39
71
39
72
39
73
39
74
39
75
39
76
40
77
40
78
40
79
40
80
40
81
40
82
40
83
40
84
39
85
39
86
39
87
39
88
39
89
39
90
39
91
39
92
39
93
38
94
38
95
38
96
38
97
38
98
38
99
38
100
38
101
38
102
38
103
38
104
38
105
38
106
38
107
38
108
38
109
38
110
38
111
38
112
39
113
39
114
39
115
39
116
39
117
39
118
39
119
39
120
40
121
40
122
40
123
40
124
40
125
40
126
40
127
40
128
40
129
40
130
40
131
40
132
40
133
40
134
40
135
40
136
40
137
41
138
41
139
41
140
41
141
41
142
41
143
41
144
42
145
42
146
42
147
42
148
42
149
42
150
42
151
42
152
42
153
42
154
42
155
42
156
42
157
42

## Calculate centroid

In [6]:
# Load the tracking data from a CSV file
df = pd.read_csv('../results/data_sperm_tracking/sperm_tracking_data_' + video_number + '_30s_15_20_01.csv')

# Calculate velocity for each track_id
df['velocity_x'] = 0.0
df['velocity_y'] = 0.0
df['speed'] = 0.0

# Frame rate of the video (frames per second)
dt = 1 / fps  # Time interval between frames

# Group by track_id and calculate velocity
for track_id, group in df.groupby('track_id'):
    # Calculate displacement (delta x and delta y)
    group['delta_x'] = group['cx'].diff()
    group['delta_y'] = group['cy'].diff()

    # Calculate velocity (pixels per second)
    group['velocity_x'] = group['delta_x'] / dt
    group['velocity_y'] = group['delta_y'] / dt

    # Calculate speed (magnitude of velocity)
    group['speed'] = (group['velocity_x']**2 + group['velocity_y']**2)**0.5
    
    # Calculate mean and maximum velocity
    group["mean_velocity"] = group['speed'].mean()
    group["max_velocity"] = group['speed'].max()
    
    df.loc[group.index, ['mean_velocity', 'max_velocity']] = group[['mean_velocity', 'max_velocity']].fillna(0)
    
    # Update the original DataFrame
    df.loc[group.index, ['velocity_x', 'velocity_y', 'speed']] = group[['velocity_x', 'velocity_y', 'speed']].fillna(0)

# Save the updated DataFrame with velocity data
df.to_csv('../results/data_sperm_track_and_velocity/sperm_tracking_with_velocity_' + video_number + '_30s_15_20_01.csv', index=False)

## Show tracking and velocitty in video

In [12]:
# Load the tracking data with velocity
df = pd.read_csv('../results/data_sperm_track_and_velocity/sperm_tracking_with_velocity_' + video_number + '_30s_15_20_01.csv')
df_class = pd.read_csv('../results/data_features_labelling/dataset_3c_11_30s_15_20_01.csv')

#df_class = df_class[df_class['sperm_id'].isin([35])]

# Open the video file
cap = cv2.VideoCapture(video_path)

# Set the slow-motion factor (1 is for normal velocity and ~0 for slow velocity)
slow_motion_factor = 1 # Adjust this value as needed
original_delay = int(1000 / fps)  # Delay in milliseconds
new_delay = int(original_delay / slow_motion_factor) # Calculate the new delay between frames

# Variables
trajectories = {}
total_class = 4

# Process the video frame by frame
frame_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get the data for the current frame
    frame_data = df[df['frame_id'] == frame_id]
    
    print(frame_data)

    # Draw sperm items
    for _, row in frame_data.iterrows():
        cx, cy = int(row['cx']), int(row['cy'])
        
        # Set sperm trajectory
        track_id = int(row['track_id'])
        if track_id not in trajectories:
            trajectories[track_id] = []
        trajectories[track_id].append((cx, cy))
        
        # Set class
        n_class = 0
        try:
            n_class = int(df_class[df_class['sperm_id']==track_id]['label'])
        except:
            continue
        
        # Define the color according to the class. EstruQcture: 2c/3c/4c
        color = (0, 0, 0)
        if n_class == 0:
            color = (0, 255, 0) # Green - Progressive/Progressive/Rapdly progressive
        elif n_class == 1:
            color = (255, 0, 0) # Blue - Non progressive/Non progressive/Slowly progressive
        elif n_class == 2:
            if total_class == 3:
                color =  (0, 0, 255)  # Red - -/-/Inmotile
            else:
                color = (0, 255, 255) # Yellow - -/Inmotile/Non progressive
        elif n_class == 3:
            color =  (0, 0, 255)  # Red - -/-/Inmotile
    
        # Draw path
        for i in range(1, len(trajectories[track_id])):
            cv2.line(frame, (int(trajectories[track_id][i - 1][0]),int(trajectories[track_id][i - 1][1])), (int(trajectories[track_id][i][0]),int(trajectories[track_id][i][1])), color, 1)
        
        # Draw rectangle for sperm head
        cv2.rectangle(frame, (int(row['xmin']), int(row['ymin'])), (int(row['xmax']), int(row['ymax'])), color, 1)
        
        # Draw id of sperm
        cv2.putText(frame, str(track_id), (cx + 10, cy), cv2.FONT_HERSHEY_PLAIN, 1.2, (255, 255, 255), 1,  cv2.LINE_AA )


    # Display the frame
    cv2.imshow('Sperm Velocity', frame)

    # Save the frame (optional)
    #cv2.imwrite(f'output/frame_{frame_id:04d}.jpg', frame)

    # Wait for the calculated delay
    if cv2.waitKey(new_delay) & 0xFF == ord('q'):
        break
    
    # Increase frame
    frame_id += 1

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()

    frame_id  video_id  track_id          cx          cy        xmin  \
0          0        11      44.0    2.740328   41.546055    0.000000   
1          0        11      43.0    8.502061  102.844986    0.443580   
2          0        11      42.0  262.690254  373.364258  252.883408   
3          0        11      41.0   17.014706   96.540581    8.994534   
4          0        11      40.0  269.605316  369.307678  259.631989   
5          0        11      39.0  133.170990  322.360077  123.515350   
6          0        11      38.0  539.738037  335.544403  530.750183   
7          0        11      37.0  468.663849  462.253998  460.744598   
8          0        11      36.0  559.259338  291.203247  550.444946   
9          0        11      35.0  442.077240  458.086578  434.050354   
10         0        11      34.0  462.476654   61.500282  454.406464   
11         0        11      33.0  467.298187  413.515839  459.124725   
12         0        11      32.0  632.267761  368.974304  624.60